In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
import numpy as np
from pathlib import Path
from typing import Any

In [ ]:
# try to read again
FILE_NAMES = [
    "red_100Hz_2024-04-01_11-29-27",
    "red_100Hz_2024-04-01_14-40-56",
    "red_50Hz_2024-04-02_09-46-52",
]
FILE_NAME = Path(FILE_NAMES[2] + ".parquet")
table = pq.read_table(FILE_NAME)
# read sample rate from filename
sample_rate_str = FILE_NAME.stem.split("_")[1]
_hz_idx = sample_rate_str.find("Hz")
sample_rate = int(sample_rate_str[:_hz_idx])
SAMPLE_RATE = sample_rate
SAMPLE_INTERVAL = 1 / SAMPLE_RATE
display(f"Sample rate: {SAMPLE_RATE} Hz", f"Sample interval: {SAMPLE_INTERVAL} s")

In [ ]:
data = table["red"].to_numpy()
data.shape

In [ ]:
import matplotlib.pyplot as plt

# sample rate is 800Hz (1.25ms per sample)
xs = np.arange(0, data.shape[0] * 1.25e-3, 1.25e-3)
# set x axis label
plt.xlabel("Time (s)")
plt.ylabel("Red LED Reading (ADC Value)")
plt.plot(xs, data)

In [ ]:
from enum import Enum, auto

THRESHOLD = 1.5e6


class Level(Enum):
    LOW = auto()
    HIGH = auto()


Segment = tuple[int, int, Level]


def segment_data(data: np.ndarray, threshold: float | int) -> list[Segment]:
    last_index = 0
    last_state = Level.HIGH if data[0] > threshold else Level.LOW
    segments: list[Segment] = []
    for i, n in enumerate(data):
        if n > threshold:
            if last_state == Level.LOW:
                segments.append((last_index, i, Level.LOW))
                last_index = i
                last_state = Level.HIGH
            else:
                continue
        else:
            if last_state == Level.HIGH:
                segments.append((last_index, i, Level.HIGH))
                last_index = i
                last_state = Level.LOW
            else:
                continue
        if i == len(data) - 1:
            segments.append((last_index, i, last_state))
    return segments

segments = segment_data(data, THRESHOLD)

In [ ]:
def segment_length(segment: Segment) -> int:
    return segment[1] - segment[0]

segment_lens = [segment_length(segment) for segment in segments]
np.percentile(segment_lens, 75)

In [ ]:
real_segments = [s for s in segments if segment_length(s) > 100]
display(real_segments)

In [ ]:
# high plot as red, low plot as blue
for segment in real_segments:
    color = "red" if segment[2] == Level.HIGH else "blue"
    plt.axvspan(segment[0] * 1.25e-3, segment[1] * 1.25e-3, color=color, alpha=0.5)

In [ ]:
import random
import plotly.express as px
import plotly.graph_objects as go
# we're only interested in the high segments
high_segments_idx = [s for s in real_segments if s[2] == Level.HIGH]
display(high_segments_idx)
high_segments = [data[s[0]:s[1]] for s in high_segments_idx]

# lucky = random.sample(high_segments, 1)[0]
# lucky_idx = random.randint(0, len(high_segments) - 1)
lucky_idx = 0
display(f"lucky index: {lucky_idx}")
# 2 might be a good one
# 1484 : 70_000
lucky = high_segments[lucky_idx]
# filter out below 1 percentile and above 99 percentile
# filtered_lucky = np.clip(lucky, np.percentile(lucky, 1),
#                          np.percentile(lucky, 99))
# TODO: maybe doing some edge detection
# like 1D canny
# I don't feel the necessity if DC offset is removed (we have different significant DC offset)
xs = np.array(range(len(lucky)))
xs_time = xs * SAMPLE_INTERVAL
# px.line(y=lucky, x=xs).show()
trace = go.Scatter(x=xs, y=lucky, mode="lines")
trace_time = go.Scatter(x=xs_time, y=lucky, mode="lines")
fig = go.Figure(data=[trace_time, trace])
# https://community.plotly.com/t/can-plotly-support-2-x-axis-and-2-y-axis-in-one-graph/38303/2
fig.update_layout(
    xaxis=dict(title="Sample Index"),
    yaxis=dict(title="Red LED Reading (ADC Value)"),
    xaxis2=dict(title="Time (s)", overlaying="x", side="top"),
)
fig.data[0].update(xaxis="x2", yaxis="y", line=dict(color="rgba(0,0,0,0)")) # type: ignore
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
from typing import Optional


workable_data:Optional[np.ndarray] = lucky
# if FILE_NAME.stem == "red_100Hz_2024-04-01_11-29-27":
#     if lucky_idx == 1:
#         workable_data = lucky[4299:-100]
#     if lucky_idx == 2:
#         workable_data = lucky[765:-50]
#     if lucky_idx == 6:
#         workable_data = lucky[1678:-200]

xs_time = np.array(range(len(workable_data))) * SAMPLE_INTERVAL # type: ignore
px.line(y=workable_data, x=xs_time).show()

In [ ]:
import heartpy as hp
from scipy.signal import butter, detrend, filtfilt, iirnotch, savgol_filter, wiener, sosfilt, sosfiltfilt, freqz, sosfreqz, ellip
from scipy.io import loadmat
from heartpy import filter_signal
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ellip.html

mat = loadmat("HR_filter_ba.2.50Hz.mat")

In [ ]:
b_2 = mat["b"].flatten()
a_2 = mat["a"].flatten()
display({
    "b": b_2,
    "a": a_2
})

In [ ]:
scipy_bp_2 = butter(1, [0.8, 5], btype="band", fs=SAMPLE_RATE, output="sos")
b_s_2, a_s_2 = butter(1, [0.8, 5], btype="band", fs=SAMPLE_RATE, output="ba")
filtered_scipy = sosfiltfilt(scipy_bp_2, workable_data)

display(f"scipy: {b_s_2.shape}, {a_s_2.shape}")
display(f"matlab 2nd order: {b_2.shape}, {a_2.shape}")
# in scipy 2nd order is the 4th order in matlab

worN = 4000

w, h = sosfreqz(scipy_bp_2, worN=worN)
w_2, h_2 = freqz(b_2, a_2, worN=worN)

fig, ax1 = plt.subplots()
ax1.set_title("Digital filter frequency response")
ax1.set_ylabel("Amplitude (ratio)")
ax1.set_xlabel("Frequency (Hz)")
ax1.grid()
ax1.set_xlim([0, 10])

ax1.plot(0.5 * SAMPLE_RATE * w / np.pi, np.abs(h), label="scipy (2nd order)")
ax1.plot(0.5 * SAMPLE_RATE * w_2 / np.pi,
         np.abs(h_2),
         label="matlab (2nd order) ellip")

# some how the matlab filter is significantly worse than scipy
ax1.legend()
plt.show()

In [ ]:
# 0.4Hz to 100Hz
# https://github.com/paulvangentcom/heartrate_analysis_python/blob/master/examples/1_regular_PPG/Analysing_a_PPG_signal.ipynb
# https://github.com/paulvangentcom/heartrate_analysis_python/blob/master/examples/5_noisy_ECG/Analysing_Noisy_ECG.ipynb
# https://github.com/paulvangentcom/heartrate_analysis_python/blob/master/docs/algorithmfunctioning.rst
# https://github.com/paulvangentcom/heartrate_analysis_python/blob/master/docs/heartrateanalysis.rst

# remove_baseline_wander is just a notch filter applied to low frequency (to remove DC offset)
# notch filter to remove DC offset
# enhance_ecg_peaks is useless
# the high pass/low pass/band pass filter here are all butterworth filter

# We will use the bandpass variant.
# we filter out frequencies below 0.8Hz (<= 48 bpm) (bpm = 60 x Hz)
# and above 3Hz (>= 180 bpm)
# Second-order sections (SOS) matrix and gain values (G) from MATLAB

# by default it only has 2nd order filter

filtered_mat = filtfilt(b_2, a_2, workable_data)
filtered_scipy = sosfiltfilt(scipy_bp_2, workable_data)

# drop the rediculously high values
# I'm not sure about the value range
filtered_scipy = np.clip(filtered_scipy, -255, 255 - 1)
filtered_mat = np.clip(filtered_mat, -255, 255 - 1)

trace_bp_matlab = go.Scatter(x=xs_time,
                               y=filtered_mat,
                               mode="lines",
                               name="Bandpass Filtered (MATLAB)")
trace_bp = go.Scatter(x=xs_time,
                      y=filtered_scipy,
                      mode="lines",
                      name="Bandpass Filtered (Scipy)")
fig = go.Figure(data=[trace_bp, trace_bp_matlab])
fig.update_layout(
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Red LED Reading (ADC Value)"),
)
fig.show()

In [ ]:
from typing import Tuple, Union
number = Union[int, float]
NDArray = np.ndarray

In [ ]:
from dataclasses import dataclass
from jaxtyping import Int, Float
from typeguard import typechecked
from numpy.lib.stride_tricks import sliding_window_view
IntArray1D = Int[NDArray, "..."]


# https://leetcode.cn/problems/sliding-window-median
# https://ipython-books.github.io/47-implementing-an-efficient-rolling-average-algorithm-with-stride-tricks/
# https://aman.ai/code/sliding-window/
# https://oi-wiki.org/ds/monotonous-queue/


# np.pad(input, (size_before, size_after), mode="edge")
# https://github.com/scipy/scipy/blob/2ecac3e596fdb458c85000e7707a8f5f46926621/scipy/ndimage/src/ni_support.c#L222
@typechecked
def extend_input(input: NDArray, size_before: int,
                 size_after: int) -> NDArray:
    """
    abcd -> abcdcba | abcd | dcbabcd
    """
    line_len = len(input)
    before_size_diff = line_len - size_before
    # [::-1] is python way to reverse (I prefer use `reversed` though)

    if size_before != 0:
        before = input[:size_before][::-1]
        if before_size_diff < 0:
            sz = abs(before_size_diff)
            before = np.concatenate([before, input[:sz][::-1]])
    else:
        before = np.array([])

    if size_after != 0:
        after_size_diff = line_len - size_after
        after = input[-size_after:][::-1]
        if after_size_diff < 0:
            sz = abs(after_size_diff)
            after = np.concatenate([after, input[:sz][::-1]])
    else:
        after = np.array([])

    return np.concatenate([before, input, after])


def rolling_mean(input: NDArray, window_size: int) -> Tuple[NDArray, number]:
    """
    input: 1D array
    window_size: window size
    """
    assert window_size > 0, "Window size must be greater than 0"
    size_1 = int(window_size / 2)
    size_2 = window_size - size_1 - 1
    padded = extend_input(input, size_1, size_2)
    var_summation = np.sum(padded[:window_size])
    output = np.zeros_like(input)
    div = var_summation / window_size
    output[0] = div

    summation = div
    # no idea how these crazy size aligns
    for i in range(window_size, len(padded)):
        var_summation += padded[i]
        var_summation -= padded[i - window_size]
        div = var_summation / window_size
        summation += div
        output[i - window_size + 1] = div

    approx_mean = summation / len(input)
    return output, approx_mean


In [ ]:
import unittest


class TestRollingMean(unittest.TestCase):

    def test_sz(self):
        input_array = np.array([1, 2, 3, 4, 5])
        r, a = rolling_mean(input_array, 3)
        self.assertEqual(len(r), len(input_array))
    
    def test_approx_mean(self):
        input_array = np.array([1, 2, 3, 4, 5, 52])
        r, a = rolling_mean(input_array, 10)
        np.testing.assert_almost_equal(a, np.mean(input_array), decimal=0)




class TestExtendInput(unittest.TestCase):

    def test_normal_case(self):
        """Test case where size_before and size_after are less than the array length"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([2, 1, 1, 2, 3, 4, 4, 3])
        np.testing.assert_array_equal(extend_input(input_array, 2, 2),
                                      expected_output)

    def test_size_before_larger(self):
        """Test case where size_before is larger than the array length"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([4, 3, 2, 1, 1, 1, 2, 3, 4, 4, 3])
        np.testing.assert_array_equal(extend_input(input_array, 5, 2),
                                      expected_output)

    def test_size_after_larger(self):
        """Test case where size_after is larger than the array length"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([2, 1, 1, 2, 3, 4, 4, 3, 2, 1, 1])
        np.testing.assert_array_equal(extend_input(input_array, 2, 5),
                                      expected_output)

    def test_both_sizes_larger(self):
        """Test case where size_before and size_after are larger than the array length"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([4, 3, 2, 1, 1, 1, 2, 3, 4, 4, 3, 2, 1, 1])
        np.testing.assert_array_equal(extend_input(input_array, 5, 5),
                                      expected_output)

    def test_empty_array(self):
        """Test case where the input array is empty"""
        input_array = np.array([])
        expected_output = np.array([])
        np.testing.assert_array_equal(extend_input(input_array, 2, 2),
                                      expected_output)

    def test_zero_sizes(self):
        """Test case where size_before and size_after are zero"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([1, 2, 3, 4])
        np.testing.assert_array_equal(extend_input(input_array, 0, 0),
                                      expected_output)

    def test_zero_size_before(self):
        """Test case where size_before is zero and size_after is larger than the array length"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([1, 2, 3, 4, 4, 3, 2, 1, 1])
        np.testing.assert_array_equal(extend_input(input_array, 0, 5),
                                      expected_output)

    def test_zero_size_after(self):
        """Test case where size_before is larger than the array length and size_after is zero"""
        input_array = np.array([1, 2, 3, 4])
        expected_output = np.array([4, 3, 2, 1, 1, 1, 2, 3, 4])
        np.testing.assert_array_equal(extend_input(input_array, 5, 0),
                                      expected_output)

    def test_single_element_array(self):
        """Test case where the input array has a single element"""
        input_array = np.array([1])
        expected_output = np.array([1, 1, 1])
        np.testing.assert_array_equal(extend_input(input_array, 1, 1),
                                      expected_output)


unittest.main(argv=[''], exit=False)


In [ ]:
@typechecked
def detect_peaks(
        hr_data: Float[NDArray, "sz"],  # noqa: F821
        rol_mean: Float[NDArray, "sz"],  # noqa: F821
        mean: number,
        ma_perc: number) -> Optional[IntArray1D]:
    """
    Detect peaks in heart rate data based on a rolling mean threshold.

    This function identifies peaks in the given heart rate data by comparing the data points
    against a rolling mean threshold. The threshold is calculated by scaling the rolling mean
    with a specified percentage.

    Parameters:
    -----------
    hr_data : NDArray
        An array containing the heart rate data points.
    rol_mean : NDArray
        An array containing the rolling mean values corresponding to each data point in hr_data.
        The length of rol_mean must be the same as hr_data.
    mean : number
        The mean value used to calculate the threshold.
    ma_perc : number
        The percentage used to scale the rolling mean. It must be a value between 0 and 2 (exclusive).
        For example, 0.1 means 10% of the peak value.

    Returns:
    --------
    NDArray
        An array containing the indices of the detected peaks in hr_data.
    """
    assert len(hr_data) == len(
        rol_mean), "Length of input data and rolling mean must be the same"
    assert ma_perc > 0, "Percentage must be greater than 0"
    assert 0 < ma_perc <= 3, "Percentage must be between 0 and 2 (0.1 means 10% of the peak value)"
    assert len(hr_data) > 0, "Input data must not be empty"

    mn = mean * ma_perc
    # this comment exists in heartpy already
    # might be an alternative way to calculate the scaled rolling mean
    #
    # r_mean = rol_mean + rol_mean * ma_perc + mn
    r_mean = rol_mean + mn

    data_ps = np.vstack((np.arange(len(hr_data)), hr_data)).T
    # grab the peak based on the scaled rolling mean
    peak_ps = data_ps[np.where(data_ps[:, 1] > r_mean)]
    if len(peak_ps) == 0:
        return None
    # not sure about this
    last_p = peak_ps[-1]

    # remove peaks that are too close to each other (peak should NOT appear
    # continuously, like the flat part of square wave)
    excl_cont_ps_ = data_ps[np.where(np.diff(peak_ps[:, 0]) > 1)]
    if len(excl_cont_ps_) == 0:
        return None
    # np.diff will return n-1 elements, add one element back (should be optional)
    excl_cont_ps = np.vstack([excl_cont_ps_, last_p])

    peak_idxs = np.array([], dtype=int)

    # find the max y-value in each interval
    # put the index of the max y-value into peak_idxs
    for p in sliding_window_view(excl_cont_ps[:, 0], 2):
        x_0 = int(p[0])
        x_1 = int(p[1])
        interval_ps = peak_ps[x_0:x_1]
        max_idx = np.argmax(interval_ps[:, 1])
        origin_idx = int(interval_ps[max_idx][0])
        peak_idxs = np.append(peak_idxs, int(origin_idx))

    return peak_idxs.astype(int)


In [ ]:
# we need 0.75s (at some sample rate)
window_size = int(0.75 / SAMPLE_INTERVAL)
r, a = rolling_mean(filtered_mat, window_size)
display(
    f"Window size: {window_size}, Approximate mean: {a}, data mean: {np.mean(filtered_mat)}, rolling mean: {np.mean(r)}"
)

In [ ]:
from warnings import warn


@dataclass
class RRInterval:
    rr: Float[NDArray, "..."]
    rr_idxs: Int[NDArray, "... 2"]


@typechecked
def preprocess_peaks_idxs(peaks_idxs: IntArray1D,
                          sample_rate: int) -> IntArray1D:
    assert sample_rate > 0, "Sample rate must be greater than 0"
    assert peaks_idxs.ndim == 1, "Peaks must be a 1D array"
    assert len(peaks_idxs) > 1, "Peaks must contain at least 2 elements"
    working_peaks = peaks_idxs
    if peaks_idxs[0] <= (sample_rate / 1000 * 150):
        working_peaks = peaks_idxs[1:]
    return working_peaks


@typechecked
def calc_rr_list(peaks_idxs: IntArray1D, sample_rate: int) -> RRInterval:
    assert peaks_idxs.ndim == 1, "Peaks must be a 1D array"
    assert len(peaks_idxs) > 1, "Peaks must contain at least 2 elements"
    rr_list = np.diff(peaks_idxs) / sample_rate * 1000
    rr_idxs = sliding_window_view(peaks_idxs, 2).astype(int)
    return RRInterval(rr_list, rr_idxs)


@typechecked
def fit_peaks(data: Float[NDArray, "..."] | Int[NDArray, "..."],
              sample_rate: int,
              hr_max: int = 190,
              hr_min: int = 48,
              rrsd_min: float = 0.1,
              rrsd_max: float = 1_000) -> Tuple[IntArray1D, RRInterval]:
    assert sample_rate > 0, "Sample rate must be greater than 0"
    assert hr_max > 0, "Maximum heart rate must be greater than 0"
    assert hr_min > 0, "Minimum heart rate must be greater than 0"
    assert hr_max > hr_min, "Maximum heart rate must be greater than minimum heart rate"
    assert len(data) > 0, "Data must not be empty"
    bl_val = np.min(data)
    workable_data = data.copy()
    if bl_val < 0:
        workable_data = workable_data + abs(bl_val)
    sample_interval = 1 / sample_rate
    # 0.75ms
    window_size = int(0.75 / sample_interval)
    r, a = rolling_mean(workable_data, window_size)

    # ma_perc_list = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 2, 3]
    ma_perc_candidate: list[float] = [0.05, 0.1, 0.25, 0.5, 0.75, 1.25, 3]
    # candidate should be sorted in ascending order
    ma_perc_candidate = sorted(ma_perc_candidate)
    peak_idxs: Optional[NDArray] = None
    rr: Optional[RRInterval] = None
    for ma_perc in ma_perc_candidate:
        _data = workable_data.copy()
        _peak_idxs = detect_peaks(_data, r, a, ma_perc)
        if _peak_idxs is None:
            warn(f"No peaks detected with ma_perc: {ma_perc}")
            continue
        bpm = len(_peak_idxs) / (len(workable_data) / sample_rate) * 60
        if bpm > hr_max or bpm < hr_min:
            warn(f"Detected heart rate is out of range: {bpm}")
            continue
        _rr = calc_rr_list(_peak_idxs, sample_rate)
        rr_std = np.std(_rr.rr)
        if rr_std < rrsd_min or rr_std > rrsd_max:
            warn(f"RR interval standard deviation is too high: {rr_std}")
            continue
        peak_idxs = _peak_idxs
        rr = _rr
        break

    if peak_idxs is None or rr is None:
        raise ValueError("No valid peak detection found")
    return peak_idxs, rr

In [ ]:
pc = np.percentile(filtered_mat, 0.1)
display(f"Percentile: {pc}")
idx_1, rr = fit_peaks(filtered_mat, SAMPLE_RATE)
# plt.plot(filtered_mat)
# plt.plot(rolling)

In [ ]:
idx_1, rr = fit_peaks(filtered_mat, SAMPLE_RATE)

fig = plt.figure(figsize=(24, 6))
plt.plot(filtered_mat)
# grey
plt.plot(r, "k", alpha=0.5)
plt.plot(idx_1, filtered_mat[idx_1], "go")
# plt.xlim([1000, 3000])
plt.show()

In [ ]:
def calc_rr(peak_list: NDArray, sample_rate: int):
    #delete first peak if within first 150ms (signal might start mid-beat after peak)
    def preprocess(peak_list: NDArray, sample_rate: int) -> NDArray:
        if len(peak_list) > 0:
            if peak_list[0] <= ((sample_rate / 1000.0) * 150):
                peak_list = np.delete(peak_list, 0)
        return peak_list

    new_peak_list = preprocess(peak_list, sample_rate)
    # RR seems in miliseconds
    rr_list = (np.diff(new_peak_list) / sample_rate) * 1000.0
    rr_indices = sliding_window_view(new_peak_list, 2)
    rr_diff = np.abs(np.diff(rr_list))
    rr_sqdiff = np.power(rr_diff, 2)
    if len(peak_list) > 0:
        rrsd = np.std(rr_list)
    else:
        rrsd = np.nan

# check_peak (by rr interval +- 30% w/ min 300ms)
# check_rr (quotient_filter)


In [ ]:
# calc_freq: whether to calculate frequency domain measures
# interp_threshold: the amplitude threshold beyond which will be checked for
# clipping. Recommended is to take this as the maximum value of the ADC with
# some margin for signal noise
# reject_segmentwise: whether to reject segments with more than 30% rejected
# beats. By default looks at segments of 10 beats at a time.

# clean_rr uses by default quotient-filtering, which is a bit aggressive.
# You can set 'iqr' or 'z-score' with the clean_rr_method flag.
from typing import Literal

CLEAN_RR_METHOD = Literal["quotient-filter", "iqr", "z-score"]
clean_rr_method: CLEAN_RR_METHOD = "iqr"
working, measures = hp.process(
    filtered_mat,
    sample_rate=SAMPLE_RATE,
    interp_clipping=False,
    clean_rr=False,
    clean_rr_method=clean_rr_method,
)

# Take into consideration that the scale for RMSSD doesn't typically exceed +/-
# 130, SDSD doesn't differ by much. This means that even a few incorrectly
# detected peaks are already introducing large measurement errors into the output
# variables. The algorithm described here is specifically designed to handle noisy
# PPG data from cheap sensors. The main design criteria was to minimise the number
# of incorrectly placed peaks as to minimise the error introduced into the output
# measures.

display(measures)
hp.plotter(working, measures, figsize=(24, 6), moving_average=True)

In [ ]:
hp.plot_breathing(working, measures, figsize=(18, 4))

In [ ]:
hp.plot_poincare(working, measures, figsize=(4, 4))